## An example of how to use a linear transformation to precondition a CG solver.

In [1]:
import autograd
import autograd.numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import time

from copy import deepcopy
import scipy as sp
from scipy.sparse.linalg import LinearOperator
from scipy import optimize
from scipy import stats

In [80]:
class Objective(object):
    def __init__(self, k, diag_min=1., diag_max=1., diag_exp=1., offdiag=0.1):
        self.k = k
        self.a = np.diag(np.linspace(diag_min, diag_max, k)  ** diag_exp) + np.full((k, k), offdiag)

        self.grad = autograd.grad(self.objective)
        self.hessian = autograd.hessian(self.objective)
        self.hessian_vector_product = autograd.hessian_vector_product(self.objective)

        self.grad_cond = autograd.grad(self.preconditioned_objective)
        self.hessian_vector_product_cond = autograd.hessian_vector_product(self.preconditioned_objective)

        self.set_preconditioner(np.identity(k))

    def set_preconditioner(self, preconditioner):
        self.preconditioner = preconditioner
        self.preconditioner_inv = np.linalg.inv(preconditioner)
        
    def objective(self, x):
        return 0.5 * np.matmul(x, np.matmul(self.a, x))

    def preconditioned_objective(self, xhat):
        x = self.get_x(xhat)
        return self.objective(x)
    
    def get_xhat(self, x):
        return np.matmul(self.preconditioner, x)

    def get_x(self, xhat):
        #return np.linalg.solve(self.preconditioner, xhat)
        return np.matmul(self.preconditioner_inv, xhat)


k = 10
obj = Objective(k, offdiag=0.1)
obj.set_preconditioner(np.diag(1. / np.diag(obj.a)))
x = np.random.random(obj.k)
xhat = obj.get_xhat(x)
print(obj.get_x(xhat) - x)
print(obj.objective(x) - obj.preconditioned_objective(xhat))

g = obj.grad(x)
hess = obj.hessian(x)
hvp = obj.hessian_vector_product(x, g)
xopt = np.full(obj.k, 0.)
print(np.linalg.norm(np.matmul(hess, g) - hvp))
print(np.linalg.norm(hess - obj.a))

g_cond = obj.grad_cond(x)
hvp_cond = obj.hessian_vector_product_cond(x, g)

KLHessVecProdLO = LinearOperator((obj.k, obj.k), lambda vec: obj.hessian_vector_product(xopt, vec))
hinv_vp, info = sp.sparse.linalg.cg(KLHessVecProdLO, g)
print(np.linalg.norm(np.linalg.solve(hess, g) - hinv_vp))


[  1.11022302e-16   1.11022302e-16   0.00000000e+00   1.11022302e-16
   1.11022302e-16   5.55111512e-17   0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.55111512e-17]
-4.4408920985e-16
0.0
0.0
6.55055105781e-16


In [84]:
k = 300
obj = Objective(k, offdiag=1.1, diag_min=2., diag_exp=2., diag_max=100.)
preconditioner = sp.linalg.sqrtm(obj.a)
#print(np.diag(np.linalg.solve(preconditioner.T, np.matmul(obj.a, np.linalg.inv(preconditioner)))))
obj.set_preconditioner(preconditioner)
x = np.random.random(obj.k)

tr_opt = optimize.minimize(
    obj.objective,
    x0=x, jac=obj.grad, hessp=obj.hessian_vector_product,
    method='trust-ncg', options={'maxiter': 500})

tr_opt_cond = optimize.minimize(
    obj.objective,
    x0=obj.get_xhat(x), jac=obj.grad_cond, hessp=obj.hessian_vector_product_cond,
    method='trust-ncg', options={'maxiter': 500})

print('\n\nTrust region:\n')
print(tr_opt.nfev)
print(tr_opt.message)
print(np.linalg.norm(tr_opt.x)) # Should be 0

print('\n\nPreconditioned:\n')
print(tr_opt_cond.nfev)
print(tr_opt_cond.message)
print(np.linalg.norm(tr_opt_cond.x)) # Should be 0




Trust region:

22
Optimization terminated successfully.
4.84559500808e-07


Preconditioned:

11
Optimization terminated successfully.
2.35739542919e-12
